# RoBERTa Model

Roberta is an improved model based on Bert. Let's see if it can has better performance.
**Since the majority of process is similar to Bert Model mention in Part_2_3, here I'll just explain the parts don't mention before.**

## 1. Load Data

In [1]:
# set the 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.10.0+cu113
True


In [3]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [4]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import gensim
import contractions
import re
import tensorflow as tf
import keras
%matplotlib inline

print("gensim: " + gensim.__version__)
print("tensorflow: " + tf.__version__)
print("keras: " + keras.__version__)

2022-11-22 20:33:55.763161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 20:34:15.205786: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64
2022-11-22 20:34:15.206156: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64
2022-11-22 20:34:15.20

gensim: 4.2.0
tensorflow: 2.11.0
keras: 2.11.0


In [5]:
# import library
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from tabulate import tabulate
from tqdm import trange
import random

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /home/Vivian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/Vivian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/Vivian/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [8]:
## ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
# load the data
df = pd.read_pickle("part2_data/cleaned_tweets.pkl")

In [10]:
df_train = df.loc[df.identification == 'train']
df_test = df.loc[df.identification == 'test']
df_train[:3]

,tweet_id,text,hashtags,emotion,identification,unigrams,remove_stopwords
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",[Snapchat],anticipation,train,"[People, who, post, ``, add, me, on, #, Snapch...",People post `` add Snapchat '' must dehydrated...
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]",sadness,train,"[@, brianklaas, As, we, see, ,, Trump, is, dan...",brianklaas see Trump dangerous freepress aroun...
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,[],fear,train,"[Now, ISSA, is, stalking, Tasha, 😂😂😂, <, LH, >]",ISSA stalking Tasha 😂😂😂 LH


In [11]:
print(df_train.groupby(['emotion']).count()['text'])
# df_train['emotion'].value_counts().plot(kind='bar')
# plt.title('Emotion class count')
# plt.xlabel("category")

emotion
anger            39867
anticipation    248935
disgust         139101
fear             63999
joy             516017
sadness         193437
surprise         48729
trust           205478
Name: text, dtype: int64


In [12]:
# condition = df_train['emotion'] == 'anger'
# df_select = df_train[condition].sample(39867)
# condition = df_train['emotion'] == 'anticipation'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'disgust'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'fear'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'joy'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'sadness'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'surprise'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])
# condition = df_train['emotion'] == 'trust'
# df_tmp = df_train[condition].sample(39867)
# df_select = pd.concat([df_select, df_tmp])

# df_train = df_select.reset_index(drop=True)
# print(df_train.groupby(['emotion']).count()['text'])

## 2. Preprocessing

### 2.1 Sentence preprocessing

1. lemmatize: to uniform the text format
2. remove stopword: since the negative terms may affect the emotion of the text, they should not be removed. Finally, I didn't remove the stopwords. The reason will be illustrated in the discussion part.
3. add hashtags: here I take the hastags into consideration by adding them to the back of the texts.

In [13]:
def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()    # lemmatization
    # keep the negative terms to maintian the semantic
    negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                        'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    # preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split() if temp not in stop_words] #lemmatization
    preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split()]
    return ' '.join([x for x in preprocessed_tokens]).strip()

In [14]:
# add hashtags to text
df_train['hashtags'] = df_train['hashtags'].apply(lambda x: ' '.join(map(str, x)))
df_test['hashtags'] = df_test['hashtags'].apply(lambda x: ' '.join(map(str, x)))

df_train["text"] = df_train["text"] + ' ' + df_train["hashtags"]
df_test["text"] = df_test["text"] + ' ' + df_test["hashtags"]

In [15]:
df_train[' text'] = df_train['text'].apply(lambda x: preprocess(x))
df_test['text'] = df_test['text'].apply(lambda x: preprocess(x))

In [16]:
text = df_train.text.values
emotion = df_train.emotion.values

### 2.2 One-hot encoding

In [17]:
# deal with label into one hot
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(emotion)
print(label_encoder.classes_)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']


In [18]:
emotion = label_encode(label_encoder, emotion)
emotion[:3]

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

### 2.3 Tokenize encoding

In [19]:
# load pretrain tokenizer
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [20]:
# get the tokens and normalize them(padding, adding mask)
def roberta_encode(input_text, tokenizer):
    token_text = tokenizer.encode_plus(input_text,
                                       add_special_tokens = True,
                                       max_length = 40,
                                       pad_to_max_length = True,
                                       return_attention_mask = True,
                                       return_tensors = 'pt')
    return token_text

In [21]:
token_id = []
attention_masks = []

for sample in text:
    encoding_dict = roberta_encode(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
emotion = torch.tensor(emotion)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Since the parameterof the model is confirmed prviously (By training a lot of times). Here I don't set the validation set.

In [22]:
# get splitting index
train_idx, val_idx = train_test_split(np.arange(len(emotion)), test_size = 0.000008, shuffle = True, stratify = emotion, random_state = 42)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], emotion[train_idx])
val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], emotion[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, sampler = RandomSampler(train_set), batch_size = 32)
validation_dataloader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = 32)

In [23]:
print('shape of emotion:', emotion.shape)
print('shape of token_id: ', token_id.shape)
print('shape of attention_mask: ', attention_masks.shape)

shape of emotion: torch.Size([1455563, 8])
shape of token_id:  torch.Size([1455563, 40])
shape of attention_mask:  torch.Size([1455563, 40])


## 3. RoBERTa Model

### 3.1 Build Model

In [ ]:
# Load the BertForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest',
                                                         num_labels = 8, output_attentions = False, 
                                                         output_hidden_states = False,
                                                        ignore_mismatched_sizes=True)

# set Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-5, eps = 1e-08)

In [25]:
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

### 3.2 Training

#### Define the training statistics

In [27]:
def b_metrics(preds, labels):
    preds = np.argmax(preds, axis = 1).flatten()
    labels = np.argmax(labels, axis = 1).flatten()
    accuracy = accuracy_score(preds, labels)
    precision = precision_score(preds, labels, average='macro')
    recall = recall_score(preds, labels, average='macro')
    f1 = f1_score(preds, labels, average='macro')
    return accuracy, precision, recall, f1

#### Train!

In [28]:
epochs = 2

for e in trange(epochs, desc = 'Epoch'):
    
    '''========== Training =========='''
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    '''========== Validation =========='''

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_f1 = []
    running_loss = 0.0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            # Forward pass
            eval_output = model(b_input_ids,
                                token_type_ids = None,
                                attention_mask = b_input_mask)

        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()


        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_f1 = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update f1 only when (tn + fp) !=0; ignore nan
        if b_f1 != 'nan': val_f1.append(b_f1)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation f1: {:.4f}\n'.format(sum(val_f1)/len(val_f1)) if len(val_f1)>0 else '\t - Validation f1: NaN')
    file_path = './model/RoBERTa_14_' + str(e)
    torch.save(model, file_path)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


	 - Train loss: 0.1928
	 - Validation Accuracy: 0.5000
	 - Validation Precision: 0.3214
	 - Validation Recall: 0.3214
	 - Validation f1: 0.2976



Epoch:  50%|█████     | 1/2 [52:50<52:50, 3170.05s/it]


	 - Train loss: 0.1688
	 - Validation Accuracy: 0.5833
	 - Validation Precision: 0.5833
	 - Validation Recall: 0.4556
	 - Validation f1: 0.4905



Epoch: 100%|██████████| 2/2 [1:35:52<00:00, 2876.40s/it]


In [29]:
torch.save(model, './model/RoBERTa_14')

### 3.3 Predict

In [30]:
model = torch.load('./model/RoBERTa_14')

In [31]:
device

device(type='cuda')

In [32]:
test_text = df_test.text.values

In [33]:
test_token_id = []
test_attention_mask = []

for t in test_text:
    encoding_dict = roberta_encode(t, tokenizer)
    test_token_id.append(encoding_dict['input_ids']) 
    test_attention_mask.append(encoding_dict['attention_mask'])
    
test_token_id = torch.cat(test_token_id, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

test_token_id = torch.unsqueeze(test_token_id, 1)
test_attention_mask = torch.unsqueeze(test_attention_mask, 1)

In [34]:
# Forward pass, calculate logit predictions
predict = []
with torch.no_grad():
    for t, m, i in zip(test_token_id, test_attention_mask, range(len(test_attention_mask))):
        output = model(t.to(device), token_type_ids = None, attention_mask = m.to(device))
        if i % 1000 == 0: print("finished:", i)
        predict.append(output.logits.cpu().numpy().flatten())
        # predict.append(np.argmax(output.logits.cpu().numpy()).flatten().item())

finished: 0
finished: 1000
finished: 2000
finished: 3000
finished: 4000
finished: 5000
finished: 6000
finished: 7000
finished: 8000
finished: 9000
finished: 10000
finished: 11000
finished: 12000
finished: 13000
finished: 14000
finished: 15000
finished: 16000
finished: 17000
finished: 18000
finished: 19000
finished: 20000
finished: 21000
finished: 22000
finished: 23000
finished: 24000
finished: 25000
finished: 26000
finished: 27000
finished: 28000
finished: 29000
finished: 30000
finished: 31000
finished: 32000
finished: 33000
finished: 34000
finished: 35000
finished: 36000
finished: 37000
finished: 38000
finished: 39000
finished: 40000
finished: 41000
finished: 42000
finished: 43000
finished: 44000
finished: 45000
finished: 46000
finished: 47000
finished: 48000
finished: 49000
finished: 50000
finished: 51000
finished: 52000
finished: 53000
finished: 54000
finished: 55000
finished: 56000
finished: 57000
finished: 58000
finished: 59000
finished: 60000
finished: 61000
finished: 62000
finis

In [35]:
y_pred = label_decode(label_encoder, predict)
tweet_id = df_test.tweet_id.tolist()

In [36]:
df = pd.DataFrame(list(zip(tweet_id, y_pred)), columns =['id', 'emotion']) 
df.to_csv("result_RoBERTa_14.csv",index=False)